# **Carregar o dataset**

In [ ]:
import pandas as pd # importando o pandas para manipularmos o dataset
import numpy as np # biblioteca para manipulação de vetores e matrizes grandes além de outras manipulações de dados de larga escala
import seaborn as sns # importando o Seaborn para visualizar o comportamento dos dados
import matplotlib.cm as mcm # biblioteca para mostrar gráficos (espeficamente uma parte para cores)
import matplotlib.pyplot as plt # importando o Matplotlib para o elbow method
import pandas_profiling

from pandas_profiling import ProfileReport # importando o pandas-profiling para fazer o profile do dataset
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV# utilizado para o split entre treinamento e teste
from sklearn.neighbors import KNeighborsRegressor # KNN para regressão
from sklearn.neighbors import KNeighborsClassifier # utilizado para treinar o KNN
from sklearn.linear_model import LinearRegression, LogisticRegression # Regressão linear, utilizado para treinar um modelo de classificação (regressão logística - apesar do nome é para problemas de classificação)
from sklearn.svm import SVR # SVM para regressão
from sklearn.decomposition import PCA # PCA como aprendizagem não-supervisionada
from sklearn.preprocessing import RobustScaler, OrdinalEncoder, OneHotEncoder # utilizado para que todas as entradas estejam na mesma escala numérica
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import * # importando todas as funções específicas de seleção de atributos do scikit-learn
from sklearn.decomposition import * # importando todas as funções específicas para a extração de atributos do scikit-learn
from sklearn.cluster import * # importando todas as funções específicas para o agrupamento
from sklearn.ensemble import * # importando vários ensembles para que possamos testá-los posteriormente
from sklearn.pipeline import Pipeline # utilizado para criar pipelines
from sklearn.metrics import f1_score, make_scorer # utilizado para calcular a performance dos pipelines
from lightgbm import LGBMClassifier # utilizado para treinar o LightGBM
from sklearn import set_config # utilizado para mostrar os passos do pipeline de forma visual
from sklearn.impute import SimpleImputer

set_config(display='diagram') # forçando para que os passos do pipeline sejam mostrados em visual

In [ ]:
#seleção do dataset
df_nba = pd.read_csv('/content/nba_stats.csv') #carrega o dataset do diretorio 
df_nba.describe() #mostra destacaria os dados
#display(df_nba.columns)

,Ano,Idade,Jogos,TrueShootingPercentage,FreeThrowRate,OffensiveWinShares,DefensiveWinShares,WinShares,FieldGoals,FieldGoalAttempts,...,TwoPointFieldGoals,TwoPointFieldGoalAttempts,TwoPointFieldGoalPercentage,EffectiveFieldGoalPercentage,FreeThrows,FreeThrowAttempts,FreeThrowPercentage,Assists,PersonalFouls,Points
count,24624.000000,24616.000000,24624.000000,24538.000000,24525.000000,24585.000000,24585.000000,24585.000000,24624.000000,24624.000000,...,24624.000000,24624.000000,24496.000000,24525.000000,24624.000000,24624.000000,23766.000000,24624.000000,24624.000000,24624.000000
mean,1992.594989,26.664405,50.837110,0.493001,0.325455,1.257307,1.227395,2.485796,195.325820,430.645752,...,178.250447,381.756782,0.445343,0.450658,102.389336,136.775219,0.719279,114.852623,116.339222,510.116350
std,17.429594,3.841892,26.496161,0.094469,0.218971,2.136256,1.269613,3.058638,188.114361,397.624715,...,179.478923,371.260335,0.099803,0.099200,113.373565,146.078918,0.141824,135.863913,84.791873,492.922981
min,1950.000000,18.000000,1.000000,0.000000,0.000000,-5.100000,-1.000000,-2.800000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1981.000000,24.000000,27.000000,0.458000,0.208000,-0.100000,0.200000,0.200000,41.000000,99.000000,...,35.000000,82.000000,0.407000,0.414000,18.000000,27.000000,0.657000,19.000000,39.000000,106.000000
50%,1996.000000,26.000000,58.000000,0.506000,0.296000,0.400000,0.800000,1.400000,141.000000,321.000000,...,122.000000,270.000000,0.456000,0.463000,63.000000,88.000000,0.743000,68.000000,109.000000,364.000000
75%,2007.000000,29.000000,75.000000,0.544000,0.400000,1.900000,1.800000,3.800000,299.000000,661.000000,...,268.000000,579.250000,0.496000,0.501000,149.000000,201.000000,0.808000,160.000000,182.000000,778.000000
max,2017.000000,44.000000,88.000000,1.136000,6.000000,18.300000,16.000000,25.400000,1597.000000,3159.000000,...,1597.000000,3159.000000,1.000000,1.500000,840.000000,1363.000000,1.000000,1164.000000,386.000000,4029.000000


# **Divisão do dataset**

In [ ]:
# criando uma cópia do dataset para fins de teste
df_nba_copy = df_nba[~pd.isnull(df_nba["WinShares"])].copy()
df_nba_copy

,Ano,Posicao,Idade,Jogos,TrueShootingPercentage,FreeThrowRate,OffensiveWinShares,DefensiveWinShares,WinShares,FieldGoals,...,TwoPointFieldGoals,TwoPointFieldGoalAttempts,TwoPointFieldGoalPercentage,EffectiveFieldGoalPercentage,FreeThrows,FreeThrowAttempts,FreeThrowPercentage,Assists,PersonalFouls,Points
0,1950,G-F,31.0,63,0.368,0.467,-0.1,3.6,3.5,144,...,144,516,0.279,0.279,170,241,0.705,176,217,458
1,1950,SG,29.0,49,0.435,0.387,1.6,0.6,2.2,102,...,102,274,0.372,0.372,75,106,0.708,109,99,279
2,1950,SF,25.0,67,0.394,0.259,0.9,2.8,3.6,174,...,174,499,0.349,0.349,90,129,0.698,140,192,438
3,1950,F,24.0,15,0.312,0.395,-0.5,-0.1,-0.6,22,...,22,86,0.256,0.256,19,34,0.559,20,29,63
4,1950,F,24.0,13,0.308,0.378,-0.5,-0.1,-0.6,21,...,21,82,0.256,0.256,17,31,0.548,20,27,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24619,2017,PF,24.0,62,0.604,0.442,3.4,2.2,5.6,253,...,253,442,0.572,0.571,133,196,0.679,99,189,639
24620,2017,C,27.0,51,0.508,0.247,0.5,0.6,1.0,78,...,78,157,0.497,0.494,22,39,0.564,42,61,178
24621,2017,C,20.0,19,0.346,0.161,-0.1,0.1,0.0,10,...,10,31,0.323,0.323,3,5,0.600,4,17,23
24622,2017,SF,22.0,44,0.503,0.181,-0.3,0.8,0.5,88,...,55,122,0.451,0.473,31,40,0.775,36,78,240


In [ ]:
# split entre treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_nba_copy.drop('WinShares', axis=1), # aqui informamos os atributos
                                                    df_nba_copy['WinShares'], # aqui informamos as labels e na mesma ordem dos atributos
                                                    test_size=0.25, # informamos a porcentagem de divisão da base. Geralmente é algo entre 20% (0.20) a 35% (0.35)
                                                    random_state=0) # aqui informamos um "seed". É um valor aleatório e usado para que alguns algoritmos iniciem de forma aleatória a sua divisão.

# **Preparação dos dados**

In [ ]:
# convertendo todas as colunas que são do tipo texto
# o handle_unknown é usado porque podem existir valores que só existem no X_test e não no X_train
# e queremos que a conversão não retorne em um erro caso isto aconteça
encoder_df1 = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1).fit(X_train)

# convertendo X_train e X_test para valores numéricos.
# não usamos o fit_transform porque precisaremos dos encoders novamente para X_test e y_test
X_train = encoder_df1.transform(X_train)
X_test = encoder_df1.transform(X_test)

In [ ]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
X_train = imp_mean.fit_transform(X_train)
X_test = imp_mean.transform(X_test)

# **Treinamento**

In [ ]:
modelo_knn = KNeighborsRegressor().fit(X_train, y_train)
modelo_knn.score(X_test, y_test)

0.6768621583035929

In [ ]:
modelo_lr = LinearRegression().fit(X_train, y_train)
modelo_lr.score(X_test, y_test)

0.95232854023798

In [ ]:
modelo_svm = SVR().fit(X_train, y_train)
modelo_svm.score(X_test, y_test)

0.8708416710358469

# **Mostrando as previsões**

In [ ]:
df_test = pd.DataFrame(X_test) 
df_test['Quality_Real'] = y_test.values
df_test['Quality_Predicao_KNN'] = modelo_knn.predict(X_test)
df_test['Quality_Predicao_Linear'] = modelo_lr.predict(X_test)
df_test['Quality_Predicao_SVM'] = modelo_svm.predict(X_test)
df_test

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,Quality_Real,Quality_Predicao_KNN,Quality_Predicao_Linear,Quality_Predicao_SVM
0,46.0,15.0,11.0,74.0,388.0,287.0,68.0,35.0,346.0,712.0,...,170.0,221.0,433.000000,262.0,246.0,879.0,5.7,4.96,5.702844,5.420953
1,34.0,19.0,8.0,80.0,418.0,287.0,112.0,28.0,679.0,1314.0,...,345.0,410.0,505.000000,251.0,165.0,1685.0,9.5,8.42,9.416411,9.484697
2,0.0,0.0,6.0,9.0,156.0,515.0,36.0,10.0,4.0,13.0,...,2.0,7.0,22.000000,1.0,17.0,10.0,0.1,-0.18,0.009982,-0.074762
3,51.0,9.0,6.0,81.0,381.0,448.0,99.0,80.0,701.0,-1.0,...,409.0,-1.0,282.000000,245.0,247.0,1753.0,13.2,10.04,12.854033,7.784729
4,47.0,12.0,16.0,2.0,345.0,0.0,37.0,9.0,2.0,4.0,...,0.0,0.0,385.965732,3.0,5.0,4.0,0.0,0.02,0.002596,0.034604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6142,32.0,12.0,8.0,81.0,343.0,203.0,53.0,47.0,618.0,1333.0,...,212.0,303.0,364.000000,560.0,317.0,1456.0,5.5,8.24,5.402374,5.729998
6143,12.0,4.0,4.0,7.0,321.0,263.0,37.0,9.0,7.0,14.0,...,1.0,4.0,11.000000,1.0,3.0,15.0,0.0,0.12,0.007495,0.005714
6144,34.0,19.0,5.0,75.0,329.0,131.0,39.0,12.0,258.0,573.0,...,66.0,88.0,414.000000,116.0,166.0,592.0,0.5,1.94,0.492753,0.940271
6145,0.0,19.0,13.0,60.0,275.0,354.0,80.0,17.0,287.0,832.0,...,260.0,314.0,492.000000,215.0,160.0,834.0,5.0,4.42,5.118090,4.351130
